In [1]:
from collections import Counter
from itertools import combinations

from Analyse_et_statistiques.stats import *
from Identification_couples_livres.extraction_couples_livres import *

def text_between_parentheses(text: str) -> list:
    """
    Récupère le contenu du texte entre parenthèse
    :param str text: chaine de caractère contenant ou non du texte entre prenthèse
    :return list[str]: contenu du text entre parenthèse
    """
    n = 1  # run at least once
    parenthese = []
    while n:
        if re.search(r'\([^()]*\)', text):
            parenthese.append(re.search(r'\([^()]*\)', text).group(0))
        text, n = re.subn(r'\([^()]*\)', '', text)  # remove non-nested/flat balanced parts
    return parenthese

# Rapport sur les données complêtes
## Par base de donnée:
### ADP
#### Global

In [2]:
g_book_ADP = rdflib.Graph()
g_book_ADP.parse("../Graphes/grapheADPLivres.rdf")
g_author_ADP = rdflib.Graph()
g_author_ADP.parse("../Graphes/grapheADPAuteurs.rdf")
g_editor_ADP = rdflib.Graph()
g_editor_ADP.parse("../Graphes/grapheADPEditeurs.rdf")

stats_books_ADP = get_ADP_stats(g_book_ADP, g_author_ADP, g_editor_ADP)
ADP_books = get_ADP_books_from_graph(g_book_ADP, g_author_ADP)

print(json.dumps(format_result(stats_books_ADP), indent=4))

{
    "total": 15469,
    "auteurs": 17266,
    "editeurs": 15469,
    "sujet_principal": 8092,
    "sujet": 9109,
    "auteurs_by_value": {
        "Collectif": 676,
        "Alain M. Bergeron": 215,
        "Jim Davis": 196,
        "Sampar": 192,
        "Michel Quintin": 173,
        "Eric S\u00e9vigny": 88,
        "BOBBIE KALMAN": 81,
        "Anne-Marie Chalifoux": 64,
        "Pierre Brignaud": 60,
        "Jean Morin": 58
    },
    "editeurs_by_value": {
        "Les \u00c9ditions Qu\u00e9bec-Livres": 2282,
        "Libre Expression": 1211,
        "Presses Aventure": 979,
        "Tr\u00e9carr\u00e9": 978,
        "VLB \u00e9diteur": 833,
        "Bayard Canada": 782,
        "Michel Quintin": 779,
        "Stank\u00e9": 759,
        "Les \u00c9ditions de l'Hexagone": 752,
        "La Courte \u00e9chelle": 734
    },
    "sujet_principal_by_value": {
        "FB": 1270,
        "WZS": 965,
        "YFB": 892,
        "JM": 688,
        "JNLB": 519,
        "DC": 417,
       

#### Commentaires:
On a 15469 livres référencés.
- On a un isbn unique par livre: ADP fait la différence entre les éditions
- On a un isbn pour chaque livre, pas d'isbn non indiqué
- sujet théma présent pour 59% des livres
- sujet théma principale présent pour 52% des livres
- description présente pour 45% des livres
- description promotionnelle présent pour 10% des livres
- editeur présent pour tout les livres
- titre présent pour tout les livres
- 75 commentaire en tout

#### Titre
Pour le titre on normalise simplement:

In [3]:
for exemple in ADP_books:
    if "(" in exemple.title_raw:
        print("titre intial:", exemple.title_raw, " titre traité: ", normalize(exemple.title_raw))
        break

titre intial: GD DETOUR (LE)  titre traité:  gd detour


#### Auteur
Pour les auteurs, on normalise en utilisant la fonction de normalisation. Il n'y a pas besoin de traiter plus loin,
car la séparation des auteurs est déjà effectuée dans le graph. On retire aussi les caractères '"' qui sont redondants

In [4]:
for exemple in ADP_books:
    good_exemple = False
    if len(exemple.authors) > 1:
        for author in exemple.authors_raw:
            if '(' in author:
                good_exemple = True
    if good_exemple:
        print("auteur(s) initial(s): ", exemple.authors_raw, "auteur(s) traité: ", exemple.authors)
        break

auteur(s) initial(s):  ['Stéphanie Milot', 'Isabelle (Dr) Rousseau-Caron'] auteur(s) traité:  ['stephanie milot', 'isabelle rousseau-caron']


#### Isbn
Pour ADP, on sait que l'isbn est unique et qu'il sont tous des isbn 13.
En revanche certains manquent le chiffre de contrôle. On se contente de le normaliser


In [5]:
for exemple in ADP_books:
    if len(exemple.isbns_raw[0].replace("\"", "")) == 12:
        print(normalize_isbn(exemple.isbns_raw[0]))
        print("exemple d'isbn avec le chiffre de controle manquant: \nisbn initial(s): ", exemple.isbns_raw, "isbn traité: ", exemple.isbns)
        break
for exemple in ADP_books:
    if len(exemple.isbns_raw[0].replace("\"", "")) == 13 and exemple.isbns_raw[0].replace("\"", "")[-1] != 'X':
        print("exemple d'isbn 13 normalisé: \nisbn initial(s): ", exemple.isbns_raw, "isbn traité: ", exemple.isbns)
        break

629048138522#
exemple d'isbn avec le chiffre de controle manquant: 
isbn initial(s):  ['"629048138522"'] isbn traité:  ['629048138522#']
exemple d'isbn 13 normalisé: 
isbn initial(s):  ['"0000000435970"'] isbn traité:  ['0000000435970']


### ILE
#### Global

In [6]:
with open("../Data/ILE/auteurs_ILE_comma_separated.csv", 'r', encoding='ISO-8859-1') as authors_ILE_file:
    csv_reader = csv.DictReader(authors_ILE_file, delimiter=',', fieldnames=[
        'uri', 'nom', 'bio', 'genres', 'site', 'pseudonyme'])
    authors_ILE_from_csv = [x for x in csv_reader]

with open("../Data/ILE/oeuvres_ILE_comma_separated.csv", 'r', encoding='ISO-8859-1') as books_ILE_file:
    csv_reader = csv.DictReader(books_ILE_file, delimiter=',', fieldnames=[
        'id', 'title', 'datePublished', 'author_uri', 'edition', 'publication_place', 'isbn'])
    books_ILE_from_csv = [x for x in csv_reader]

stats_books_ILE = get_stats_from_csv_reader(books_ILE_from_csv)
stats_authors_ILE = get_stats_from_csv_reader(authors_ILE_from_csv)
ILE_books = get_ILE_books_from_csv(books_ILE_from_csv, authors_ILE_from_csv)

Livres:

In [7]:
print(json.dumps(format_result(stats_books_ILE), indent=4))

{
    "total": 27109,
    "id": 27109,
    "id_by_value": {
        "23967": 1,
        "23770": 1,
        "25213": 1,
        "21714": 1,
        "25095": 1,
        "25542": 1,
        "19766": 1,
        "18901": 1,
        "19764": 1,
        "21715": 1
    },
    "title": 27109,
    "title_by_value": {
        "Maria Chapdelaine": 75,
        "Les Anciens Canadiens": 21,
        "Agaguk": 19,
        "Po\u00c3\u00a8mes": 19,
        "Le Survenant": 18,
        "L'Oubli\u00c3\u00a9": 16,
        "Une de perdue, deux de trouv\u00c3\u00a9es": 15,
        "Un homme et son p\u00c3\u00a9ch\u00c3\u00a9": 15,
        "Adagio": 15,
        "Les insolences du fr\u00c3\u00a8re Untel": 14
    },
    "datePublished": 27109,
    "datePublished_by_value": {
        "2009": 857,
        "2006": 781,
        "2007": 762,
        "2008": 749,
        "2000": 709,
        "2005": 701,
        "2010": 698,
        "1997": 697,
        "1999": 697,
        "2001": 691
    },
    "author_uri": 27109,


Auteurs:

In [8]:
print(json.dumps(format_result(stats_authors_ILE), indent=4))

{
    "total": 1791,
    "uri": 1791,
    "uri_by_value": {
        "http://recif.litterature.org/recherche/ecrivains/.-nathanael-825/": 1,
        "http://recif.litterature.org/recherche/ecrivains/acquelin-jose-27/": 1,
        "http://recif.litterature.org/recherche/ecrivains/adam-monique-1679/": 1,
        "http://recif.litterature.org/recherche/ecrivains/agnant-marie-celie-885/": 1,
        "http://recif.litterature.org/recherche/ecrivains/ajoul-hassan-1577/": 1,
        "http://recif.litterature.org/recherche/ecrivains/al-yasiri-issa-hassan-1905/": 1,
        "http://recif.litterature.org/recherche/ecrivains/alain-sonia-1345/": 1,
        "http://recif.litterature.org/recherche/ecrivains/alarie-donald-28/": 1,
        "http://recif.litterature.org/recherche/ecrivains/alavo-yves-1633/": 1,
        "http://recif.litterature.org/recherche/ecrivains/albert-michel-1262/": 1
    },
    "nom": 1791,
    "nom_by_value": {
        "B\u00c3\u00a9langer\\, Nicole": 2,
        "Boucher\\, Den

#### Commentaires
27109 livres au total
- 5931 isbn vide : soit 21%
- 405 isbn sont composé de seulement de texte entre parenthèse
exemple d'isbn

#### titre
Pour le titre on normalise simplement
On peut remaquer quelques problèmes, par exemples nous n'avons pas le titre
pour certains livres

In [9]:
for exemple in ILE_books:
    if "(" in exemple.title_raw:
        print("exemple simple: titre intial:", exemple.title_raw, " titre traité: ", exemple.title)
        break
for exemple in ILE_books:
    if not exemple.title:
        print("titre non existant: titre intial:", exemple.title_raw, " titre traité: ", exemple.title)
        break

exemple simple: titre intial: Absence Where As (Claude Cahun and the Unopened Book)  titre traité:  absence where as
titre non existant: titre intial: (Sainte-Famille)  titre traité:  


#### auteur
Pour les auteurs, on normalise en utilisant la fonction de normalisation. Il n'y a pas besoin de traiter plus loin,
car la séparation des auteurs est déjà effectuée dans le graph. On retire aussi les caractères '"' qui sont redondants

In [10]:
for exemple in ILE_books:
    good_exemple = False
    for author in exemple.authors_raw:
        if '-' in author:
            good_exemple = True
    if good_exemple:
        print("auteur(s) initial(s): ", exemple.authors_raw, "auteur(s) traité: ", exemple.authors)
        break

auteur(s) initial(s):  [' Marie-CÃ©lie Agnant\\'] auteur(s) traité:  ['marie-celie agnant']


#### isbn
Pour ILE, on sait que l'isbn peut être multiple et de forme variée
(séparé par des caractères: "-", et suivit de parenthèses), une première division s'est faite dans le graph
mais cela demande une deuxième passe, notament pour les séparateurs "|" et ";". On normalise par la suite.
On peut remarque que certains livres ont une liste conséquente d'isbn

In [11]:
for exemple in ILE_books:
    found = False
    for isbn in exemple.isbns_raw:
        if len(isbn.replace("\"", "").replace("-", "")) == 13 and isbn.replace("\"", "")[-1] != 'X':
            found = True
    if found:
        print("exemple d'isbn 13 normalisé: \nisbn initial(s): ", exemple.isbns_raw, "isbn traité: ", exemple.isbns)
        break

for exemple in ILE_books:
    found = False
    for isbn in exemple.isbns_raw:
        if len(isbn.replace("\"", "").replace("-", "")) == 10 and isbn.replace("\"", "")[-1] != 'X':
            found = True
    if found:
        print("exemple d'isbn 10 normalisé: \nisbn initial(s): ", exemple.isbns_raw, "isbn traité: ", exemple.isbns)
        break

for exemple in ILE_books:
    good_example = False
    for isbn in exemple.isbns_raw:
        for isbn in exemple.isbns_raw:
            if "|" in isbn:
                good_example = True
    if good_example:
        print("exemple d'isbn séparable par '|': ", str(exemple.isbns_raw)[0:100] + "...", "isbn traité: ", str(exemple.isbns)[0:10] + "...")
        print('nb d\'isbn: ', len(exemple.isbns_raw))
        break

for exemple in ILE_books:
    if len(exemple.isbns) > 10:
        print("exemple de livre avec plus de 10 isbn \nisbn initial(s): ", str(exemple.isbns_raw)[0:100] + "...", "isbn traité: ", str(exemple.isbns)[0:10] + "...")
        print('nb d\'isbn: ', len(exemple.isbns))
        break

exemple d'isbn 13 normalisé: 
isbn initial(s):  ['978614419332\n'] isbn traité:  ['978614419332#']
exemple d'isbn 10 normalisé: 
isbn initial(s):  ['276214713\n'] isbn traité:  ['276214713']
exemple d'isbn séparable par '|':  ['978-2-89428-937-2 (br.)|2-89428-937-5\n']... isbn traité:  ['97828942...
nb d'isbn:  1


Nous allons maintenant analyser le nombre d'isbn par livres

In [12]:
exemple = {}
compte = {}
compte_minus = {"sans":0, "avec":0}

for isbn_ls in stats_books_ILE["isbn_by_value"]:
    isbn_ls = re.split("[|;,]", isbn_ls)
    if len(isbn_ls) not in compte:
        compte[len(isbn_ls)] = 0
    compte[len(isbn_ls)] += 1
    for isbn in isbn_ls:
        if isbn:
            if "-" in isbn:
                compte_minus["avec"] += 1
            else:
                compte_minus["sans"] += 1
            if "(" in isbn:
                txt_parentheses = text_between_parentheses(isbn)
                for ex in txt_parentheses:
                    if ex not in exemple:
                        exemple[ex] = 0
                    exemple[ex] += 1
exemple = [(k, v) for k, v in sorted(exemple.items(), key=lambda item: item[1], reverse=True)]
compte = [(k, v) for k, v in sorted(compte.items(), key=lambda item: item[0], reverse=False)]
for ex, nb in exemple[:30]:
    print("exemple de texte entre parenthèses dans les isbns: ", ex, ": ", nb)

print("\nNombre d'isbn par livre")
for cpt, nb in compte:
    print(nb, " livres ont ", cpt, " isbns")
print("\nNombre d'isbn avec séparation par block (ex: '2-7644-0049-7'): ", compte_minus["avec"], ' et sans: ', compte_minus["sans"])

print("\nTest de récupération des editeurs a partir des isbns et inversement")
isbn_editor_count = {"edition and isbn": 0, "isbn": 0, "edition": 0, "None": 0}
isbn_editor_by_value = {"edition and isbn": {}, "isbn": {}, "edition": {}}
for book in books_ILE_from_csv:
    if normalize(book["edition"]) and text_between_parentheses(book["isbn"]):
        isbn_editor_count["edition and isbn"] += 1
        if book["edition"] + " " + book["isbn"] not in isbn_editor_by_value["edition and isbn"]:
            isbn_editor_by_value["edition and isbn"][book["edition"] + " " + book["isbn"]] = 0
        isbn_editor_by_value["edition and isbn"][book["edition"] + " " + book["isbn"]] += 1
    elif text_between_parentheses(book["isbn"]):
        if book["isbn"] not in isbn_editor_by_value["isbn"]:
            isbn_editor_by_value["isbn"][book["isbn"]] = 0
        isbn_editor_by_value["isbn"][book["isbn"]] += 1
        isbn_editor_count["isbn"] += 1
    elif normalize(book["edition"]):
        if book["edition"] not in isbn_editor_by_value["edition"]:
            isbn_editor_by_value["edition"][book["edition"]] = 0
        isbn_editor_by_value["edition"][book["edition"]] += 1
        isbn_editor_count["edition"] += 1
    else:
        isbn_editor_count["None"] += 1
print(json.dumps(format_result(isbn_editor_by_value), indent=4))

exemple de texte entre parenthèses dans les isbns:  (br.) :  8892
exemple de texte entre parenthèses dans les isbns:  (rel.) :  648
exemple de texte entre parenthèses dans les isbns:  (vol. 1) :  147
exemple de texte entre parenthèses dans les isbns:  (vol. 2) :  121
exemple de texte entre parenthèses dans les isbns:  (t. 1) :  88
exemple de texte entre parenthèses dans les isbns:  (rel. Ã  spirale) :  72
exemple de texte entre parenthèses dans les isbns:  (vol. 3) :  58
exemple de texte entre parenthèses dans les isbns:  (Ãcrits des Forges) :  57
exemple de texte entre parenthèses dans les isbns:  (LemÃ©ac) :  56
exemple de texte entre parenthèses dans les isbns:  (Actes sud) :  55
exemple de texte entre parenthèses dans les isbns:  (br) :  53
exemple de texte entre parenthèses dans les isbns:  (t. 2) :  52
exemple de texte entre parenthèses dans les isbns:  (t. 3) :  31
exemple de texte entre parenthèses dans les isbns:  (cart.) :  25
exemple de texte entre parenthèses dans les isbn

- On peut voir que l'on peut pas récupérerl'information de l'éditeur à partir des parenthèses des isbns, car si l'editeur est entre parenthèse, le champs "éditeur" n'est pas vide
- certain livres ont plusieur isbn: 793 livres ont 2 isbn, 87 en ont 3, etc....
- les informations entre parenthèses sont multiples: 
    - "br." (8892)
    - "rel." (648)
    - "vol." 
    - "t."
    - editeurs
- les autres informations sont prinicpalements complêtes, pas de problème majeur.

### Dépot-légal (Donnée originale, avant changement)
#### Global

In [13]:
g_book_DL = rdflib.Graph()
g_book_DL.parse("../Graphes/grapheDepotLegal.rdf")
stats_books_DL = get_depot_legal_stats_from_graph(g_book_DL)
stats_books_DL_csv = get_depot_legal_stats()
DL_books = get_depot_legal_books_from_graph(g_book_DL)
DL_authors, _ = generate_all_authors(DL_books, save=False)

Depuis le graph:

In [14]:
print(json.dumps(format_result(stats_books_DL), indent=4))

{
    "total": 56962,
    "auteurs": 57499,
    "editeurs": 56962,
    "auteurs_by_value": {
        " ": 14508,
        "Gilles Tibo": 203,
        "Robyn Bryant": 157,
        "Val\u00e9rie M\u00e9nard": 134,
        "Dominique Jolin": 134,
        "Alain M. Bergeron": 130,
        "Richard Petit": 113,
        "Laurent Chabin": 112,
        "Philip Amsel": 107,
        "Lucie Papineau": 104
    },
    "editeurs_by_value": {
        "Courte \u00c9chelle": 1705,
        "Lem\u00e9ac \u00e9diteur": 1663,
        "\u00c9ditions H\u00e9ritage": 1655,
        "\u00c9ditions AdA": 1500,
        "\u00c9ditions Hurtubise": 1470,
        "\u00c9ditions Qu\u00e9bec Am\u00e9rique, QA International": 1428,
        "\u00c9ditions Phidal": 1316,
        "Dominique et compagnie": 1161,
        "\u00c9ditions du Bor\u00e9al": 1007,
        "Presses Aventure": 937
    },
    "http://www.w3.org/1999/02/22-rdf-syntax-ns#type": 56962,
    "http://www.w3.org/1999/02/22-rdf-syntax-ns#type_by_value": {
   

Depuis le csv:

In [15]:
print(json.dumps(format_result(stats_books_DL_csv), indent=4))

{
    "total": 396470,
    "ID_DEPOT": 396470,
    "ID_DEPOT_by_value": {
        "2": 1,
        "6": 1,
        "7": 1,
        "13": 1,
        "23": 1,
        "26": 1,
        "27": 1,
        "30": 1,
        "32": 1,
        "33": 1
    },
    "TITRE_PUBLICATION": 396470,
    "TITRE_PUBLICATION_by_value": {
        "HerbArt : cours en herboristerie": 168,
        "Nouvelles histoires dr\u00f4les / texte, Jeanne Olivier": 90,
        "Programme d'\u00e9lectronique : technicien de premier niveau / [Volta Ramirez... et al.]": 79,
        "Profils socio-\u00e9conomiques des districts \u00e9lectoraux de la ville de Montr\u00e9al": 51,
        "Profil socio-\u00e9conomique : district \u00e9lectoral": 51,
        "Secr\u00e9tariat : guide d'apprentissage": 50,
        "Histoires dr\u00f4les / texte, Dominique Chauveau": 48,
        "Electronics program : level one technician": 47,
        "Carrosserie : guide d'apprentissage": 46,
        "Comptabilit\u00e9 : guide d'apprentissage": 40

#### Commentaires pour les stats du graph

- 56962 livres au total
- 57499 auteurs dont 14508 vide soit 25%
- tous ou presque tous les livres ont un nom, un type, une langue, une date de publication, et un isbn

#### titre
Pour le titre on normalise simplement
On peut remaquer quelques problèmes, par exemples nous n'avons pas le titre
pour certains livres

In [16]:
for exemple in DL_books:
    if "(" in exemple.title_raw and exemple.title:
        print("exemple simple: titre intial:", exemple.title_raw, " titre traité: ", exemple.title)
        break
for exemple in DL_books:
    if not exemple.title:
        print("titre non existant: titre intial:", exemple.title_raw, " titre traité: ", exemple.title)
        break

exemple simple: titre intial: Une virée américaine (version du 2017-01-10)   titre traité:  une viree americaine
titre non existant: titre intial: (K)  titre traité:  


#### auteur
Pour les auteurs, on normalise en utilisant la fonction de normalisation. Il n'y a pas besoin de traiter plus loin,
car la séparation des auteurs est déjà effectuée dans le graph. On retire aussi les caractères '"' qui sont redondants
En revanche, Dépot légal fait la différence entre le nom de famille et le prénom, ce que ne font pas les autres bases
de donnée. On va donc simplement concaténer les chaines de caractères avec un espace entre les noms et prénoms

In [17]:
for exemple in DL_books:
    good_exemple = False
    if len(exemple.authors) > 1:
        for author in exemple.authors:
            if '-' in author:
                good_exemple = True
    if good_exemple:
        print("auteur(s) initial(s): ", exemple.authors_raw, "auteur(s) traité: ", exemple.authors)
        break

auteur(s) initial(s):  ['"Marie-Claude" "Martel"', '"Anik" "Lessard"'] auteur(s) traité:  ['marie-claude martel', 'anik lessard']


### isbn
Pour Dépot legal, on sait que l'isbn peut être multiple et de forme variée,
 mais la séparation s'est faite dans la création du graph. On se contente de normaliser
 On peut remarque que certains livres ont une liste conséquente d'isbn

In [18]:
for exemple in DL_books:
    good_exemple = False
    for isbn in exemple.isbns_raw:
        if len(isbn.replace("\"", "")) == 13 and isbn.replace("\"", "")[-1] != 'X':
            good_exemple = True
    if good_exemple:
        print("exemple d'isbn 13 normalisé: \nisbn initial(s): ", exemple.isbns_raw, "isbn traité: ", exemple.isbns)
        break
for exemple in DL_books:
    good_exemple = False
    for isbn in exemple.isbns_raw:
        if len(isbn.replace("\"", "")) == 10 and isbn.replace("\"", "")[-1] != 'X':
            good_exemple = True
    if good_exemple:
        print("exemple d'isbn 10 normalisé: \nisbn initial(s): ", exemple.isbns_raw, "isbn traité: ", exemple.isbns)
        break
for exemple in DL_books:
    good_exemple = False
    for isbn in exemple.isbns_raw:
        if len(exemple.isbns_raw) > 60:
            good_exemple = True
    if good_exemple:
        print("exemple de livre avec plus de 60 isbn \nisbn initial(s): ", str(exemple.isbns_raw)[0:100] + "...", "isbn traité: ", str(exemple.isbns)[0:10] + "...")
        print('nb d\'isbn: ', len(exemple.isbns_raw))
        break

exemple d'isbn 13 normalisé: 
isbn initial(s):  ['"9782924022795"'] isbn traité:  ['9782924022795']
exemple d'isbn 10 normalisé: 
isbn initial(s):  ['"2894502494"'] isbn traité:  ['9782894502494']
exemple de livre avec plus de 60 isbn 
isbn initial(s):  ['"9782896600984"', '"2895433283"', '"2895433305"', '"9782896605248"', '"2895431442"', '"2895433089"... isbn traité:  ['97828966...
nb d'isbn:  65


On peut essayer de quantifier les croissements des isbns

In [19]:
print("isbn en commun pour plusieurs livres")
already_printed = []
isbn_dict = {}
for book in DL_books:
    for isbn in book.isbns:
        try:
            isbn_dict[isbn].append(book)
        except KeyError:
            isbn_dict[isbn] = [book]
i = 0
for isbn, books in isbn_dict.items():
    counter = Counter([book.title for book in books])
    already_printed_test = False
    for name in counter.values():
        if name in already_printed:
            already_printed_test = True
    if not already_printed_test:
        for name in counter.values():
            already_printed.append(name)
        print("on retrouve l'isbn: ", isbn, " dans les livres: ", [str(name) + " x " + str(ct) for name, ct in counter.items()])
        i += 1
    if i > 10:
        break
print("\nnombre d'isbn par livre:")
title_by_max_nb_isbn = {}
for book in DL_books:
    if book.title not in title_by_max_nb_isbn:
        title_by_max_nb_isbn[book.title] = len(book.isbns)
    else:
        title_by_max_nb_isbn[book.title] = max(len(book.isbns), title_by_max_nb_isbn[book.title])
for title, nb_isbn in sorted(title_by_max_nb_isbn.items(), key=lambda x: x[1], reverse=True)[:10]:
    print("pour le livre: ", title, " le nombre d'isbn est: ", nb_isbn)
    
print("\nnombre d'auteurs différents:")
for book in sorted(DL_books, key= lambda x:len(x.authors), reverse=True)[:10]:
    print("pour le livre: ", book.title, " le nombre d'auteurs est: ", len(book.authors))

print("\nAttribution d'auteurs manquants par isbn en commun avec d'autre livres:")
counter = 0
for book in DL_books:
    if not book.authors:
        for isbn, subj_isbn in isbn_dict.items():
            if book.id in subj_isbn:
                test = [i for i in subj_isbn if i!=book.id]
                for subj_i in test:
                    if subj_i in [book.id for book in DL_books]:
                        for book_i in DL_books:
                            if subj_i == book_i.id:
                                print('Ces deux livres ont le même auteur car ils ont un isbn en commun:')
                                print(book)
                                print(book_i)
                                counter += 1
                            
    if counter > 20:
        break

isbn en commun pour plusieurs livres
on retrouve l'isbn:  9782924022795  dans les livres:  ['l ivre muse x 1']
on retrouve l'isbn:  9782924022061  dans les livres:  ['l appel de mes nuits x 2']
on retrouve l'isbn:  9782896864416  dans les livres:  ['le journal d alice x 6']
on retrouve l'isbn:  9782922792508  dans les livres:  ['pierrot et le village des fous x 7']
on retrouve l'isbn:  9782922145689  dans les livres:  ['la tapisserie de fionavar x 3']
on retrouve l'isbn:  9782890747739  dans les livres:  ['les chevaliers d emeraude x 12']
on retrouve l'isbn:  9782923672007  dans les livres:  ['l heritiere de salem x 4']
on retrouve l'isbn:  9782896511464  dans les livres:  ['rock rose x 13']
on retrouve l'isbn:  9782897031046  dans les livres:  ['le blogue de namaste x 14']
on retrouve l'isbn:  9782762572940  dans les livres:  ['histoires droles x 48']
on retrouve l'isbn:  9782896675562  dans les livres:  ['les 5 derniers dragons x 8']

nombre d'isbn par livre:
pour le livre:  garfield

- on peut donc combler certain problème d'auteur manquant en comparant les isbns
- certains livres ont un nombre considérable d'isbn, car les tomes sont regroupé ensembles.
- le nombre d'auteur n'est pas abérants

### Hurtubise

#### Global

In [20]:
with open("../Data/Hurtubise/Exportation-Hurtubise.csv", "r", encoding='ISO-8859-1') as books_Hurtubise_file:
    csv_reader = csv.DictReader(books_Hurtubise_file, delimiter=',', fieldnames=[
        "Editeur", "ISBN Papier", "ISBN PDF", "ISBN epub", "Titre", "Sous - titre", "Titre de la serie",
        "Contributeurs", "Contributeur(premier)", "Langue", "Langue Origine", "Resume", "Nombre de pages",
        "Date de parution", "Annee de parution", "Sujet  THEMA principal", "Sujet THEMA",
        "Quantificateur Georaphique", "Quantificateur de langue", "Quantificateur Historique", "Niveau soclaire FR",
        "Niveau scolaire QC", "Cycle scolaire FR", "Niveau de lecture", "Echele CECR", "Quantificateur d'interet",
        "Quantificateur d'age", "Quantificateur de style", "Classification Editoriale", "Mots cles"

    ])
    Hurtubise_books = get_Hurtubise_books(csv_reader)
stats_books_Hurtubise = get_Hurtubise_Stats()
print(json.dumps(format_result(stats_books_Hurtubise), indent=4))

{
    "total": 1404,
    "Editeur": 1404,
    "Editeur_by_value": {
        "\u00c9ditions Hurtubise": 1313,
        "": 91
    },
    "ISBN Papier": 1404,
    "ISBN Papier_by_value": {
        "": 94,
        "9782897815189": 1,
        "9782897815196": 1,
        "9782897815202": 1,
        "9782897815219": 1,
        "9782897814847": 1,
        "9782897815356": 1,
        "9782897814878": 1,
        "9782897812980": 1,
        "9782897814700": 1
    },
    "ISBN PDF": 1404,
    "ISBN PDF_by_value": {
        "": 570,
        "9782896473304": 3,
        "9782896473311": 3,
        "9782896473328": 3,
        "9782896473335": 3,
        "9782896475926": 3,
        "9782896475933": 3,
        "9782896475940": 3,
        "9782896474042": 3,
        "9782896474059": 3
    },
    "ISBN epub": 1404,
    "ISBN epub_by_value": {
        "": 855,
        "9782896476503": 3,
        "9782896476510": 3,
        "9782896476527": 3,
        "9782896476534": 3,
        "9782897814670": 3,
        

#### Commentaire

- 91 livres sont totalement vides

Champs vides:
- total de 1404 livres
- 94 isbn des papiers sont vides soit 7%
- 570 isbn des PDF sont vides soit 40%
- 855 des isbn des e-pub sont vides soit 60%
- 91 livres n'ont pas de titre principal soit 6%
- 711 livres n'ont pas de sous-titre  soit 50%
- 961 livres n'ont pas de titre de la série soit 68%
- 91 contributeur et contributeur premier sont manquant soit 6%
- 92 livres n'ont pas de langue soit 6%
- 141 livres n'ont pas de langue d'origine soit 10%
- 103 livres n'ont pas de résumé soit 7%
- 100 livres n'ont pas de nombre de page soit 7%
- 615 livres n'ont pas de date de parution soit 43%
- 98 livres n'ont pas d'année de parution soit 7%
- 91 livres n'ont pas de sujet THEMA principale soit 6%
- 494 livres n'ont pas de sujet THEMA soit 35 %
- 893 livre n'ont pas de quantificateur géographique soit 64%
- 1369 livres n'ont pas de quantificateur de langue soit 97%
- 1024 livres n'ont pas de quantificateur historique soit 72%
- aucun livre n'ont de niveau scolaire FR
- seulement 4 livres ont un niveau scolaire QC
- aucun livre n'ont de niveau de lecture
- aucun livre n'ont  d'echele CECR
- 1337 livres n'ont pas de quantificateur d'interet soit 95%
- 943 livres n'ont pas de quantificateur d'age soit 67%
- seulement 6 livres ont un quantificateur de style
- 1275 livre n'ont pas de classification éditoriale soit 90%
- 929 livres n'ont pas de Mots-clefs soit 66%

### titre
Pour le titre on normalise simplement
On peut remaquer quelques problèmes, par exemple nous n'avons pas le titre
pour certains livres (les 91 livres sans aucune information)

In [21]:
for exemple in Hurtubise_books:
    if "(" in exemple.title_raw and exemple.title:
        print("exemple simple: titre intial:", exemple.title_raw, " titre traité: ", exemple.title)
        break

exemple simple: titre intial: Selfies et sushis (coffret)  titre traité:  selfies et sushis


### auteur
Pour les auteurs, on normalise en utilisant la fonction de normalisation. Il n'y a pas besoin de traiter plus loin,
car la séparation des auteurs est déjà effectuée dans le graph. On peut remarquer que beaucoup d'information
sur le status des auteurs est contenue dans les parenthèses

In [22]:
for exemple in Hurtubise_books:
    good_exemple = False
    if len(exemple.authors) > 1:
        for author in exemple.authors:
            if '-' in author:
                good_exemple = True
    if good_exemple:
        print("auteur(s) initial(s): ", exemple.authors_raw, "auteur(s) traité: ", exemple.authors)
        break

auteur(s) initial(s):  ['David Skuy (De (auteur))', ' Michel Rudel-Tessier (Traduit par)'] auteur(s) traité:  ['david skuy', 'michel rudel-tessier']


Commentaire entre parenthèse pour les auteurs:

In [23]:
exemple = {}
for isbn, cpt in stats_books_Hurtubise["Contributeurs_by_value"].items():
    if isbn:
        if "(" in isbn:
            txt_parentheses = text_between_parentheses(isbn)
            for ex in txt_parentheses:
                if ex not in exemple:
                    exemple[ex] = 0
                exemple[ex] += 1
exemple = [(k, v) for k, v in sorted(exemple.items(), key=lambda item: item[1], reverse=True)]
for ex, nb in exemple:
    print("exemple de texte entre parenthèses dans les isbns: ", ex, ": ", nb)

exemple de texte entre parenthèses dans les isbns:  (De ) :  594
exemple de texte entre parenthèses dans les isbns:  (auteur) :  462
exemple de texte entre parenthèses dans les isbns:  (Traduit par) :  112
exemple de texte entre parenthèses dans les isbns:  (Coordination éditoriale de) :  14
exemple de texte entre parenthèses dans les isbns:  (Illustré par) :  7
exemple de texte entre parenthèses dans les isbns:  (Préface de) :  4
exemple de texte entre parenthèses dans les isbns:  (photographe) :  3
exemple de texte entre parenthèses dans les isbns:  (Photographies de) :  2
exemple de texte entre parenthèses dans les isbns:  (dit Larose) :  1
exemple de texte entre parenthèses dans les isbns:  (Texte de) :  1


### isbn
Pour Hurtubise, on sait que l'isbn peut être multiple, mais toujours de taille 13. On normalise donc simplement.
On peut remarque que certains livres ont plusieurs isbns, mais pas plus de 3

In [24]:
for exemple in Hurtubise_books:
    good_exemple = False
    for isbn in exemple.isbns_raw:
        if len(isbn.replace("\"", "")) == 13 and isbn.replace("\"", "")[-1] != 'X':
            good_exemple = True
    if good_exemple:
        print("exemple d'isbn 13 normalisé: \nisbn initial(s): ", exemple.isbns_raw, "isbn traité: ", exemple.isbns)
        break
for exemple in Hurtubise_books:
    if len(exemple.isbns_raw) > 2:
        print("exemple de livre avec plus de 60 isbn \nisbn initial(s): ", str(exemple.isbns_raw)[0:100] + "...", "isbn traité: ", str(exemple.isbns)[0:10] + "...")
        print('nb d\'isbn: ', len(exemple.isbns_raw))
        break

exemple d'isbn 13 normalisé: 
isbn initial(s):  ['9782897815189', '9782896473304', '9782896476503'] isbn traité:  ['9782897815189', '9782896473304', '9782896476503']
exemple de livre avec plus de 60 isbn 
isbn initial(s):  ['9782897815189', '9782896473304', '9782896476503']... isbn traité:  ['97828978...
nb d'isbn:  3


On veut compter le nombre d'isbn des livres:

In [25]:
counter_isbn = Counter([len(book.isbns) for book in Hurtubise_books])
for key, value in dict(sorted(counter_isbn.items(), key=lambda x: x[1], reverse=True)).items():
    print(value, " livres ont ", key, " isbns")

543  livres ont  3  isbns
474  livres ont  1  isbns
295  livres ont  2  isbns
1  livres ont  0  isbns


###Babelio

####Global

In [26]:
with open('../Data/Babelio/babelio_livres.json', 'r') as f:
    Babelio_books_data = json.load(f, cls=BookJSONDecoder, encoding="utf-8")

with open('../Data/Babelio/babelio_auteurs.json', 'r') as f:
    Babelio_authors_data = json.load(f, cls=BookJSONDecoder, encoding="utf-8")

stats_book = get_babelio_stats_from_items(Babelio_books_data)
stats_author = get_babelio_stats_from_items(Babelio_authors_data)

Babelio_books = get_Babelio_books(Babelio_books_data)

Livres:

In [27]:
print(json.dumps(format_result(stats_book), indent=4))

{
    "total": 5553,
    "url": 5553,
    "url_by_value": {
        "https://www.babelio.com/livres/Sevigny-Sans-terre/887589": 1,
        "https://www.babelio.com/livres/Leroux-Madame-Victoria/768705": 1,
        "https://www.babelio.com/livres/Tremblay-Hotel-Bristol-New-York-N-Y/160254": 1,
        "https://www.babelio.com/livres/Tremblay-Le-passage-oblige/244745": 1,
        "https://www.babelio.com/livres/Beaudoin-Hadassa/124739": 1,
        "https://www.babelio.com/livres/Senecal-Quinze-minutes/459141": 1,
        "https://www.babelio.com/livres/Dompierre-Un-petit-pas-pour-lhomme/46346": 1,
        "https://www.babelio.com/livres/Boulerice-Javotte/417516": 1,
        "https://www.babelio.com/livres/Tremblay-Le-coeur-decouvert/37083": 1,
        "https://www.babelio.com/livres/Proulx-Ce-quil-reste-de-moi/708379": 1
    },
    "titre": 5553,
    "titre_by_value": {
        " Po\u00e8mes ": 4,
        " Po\u00e9sies compl\u00e8tes ": 3,
        " Nous ": 3,
        " Po\u00e8mes choi

Auteurs

In [28]:
print(json.dumps(format_result(stats_author), indent=4))

{
    "total": 2768,
    "url": 2768,
    "url_by_value": {
        "https://www.babelio.com/auteur/Simon-Roy/326355": 1,
        "https://www.babelio.com/auteur/Milena-Babin/302343": 1,
        "https://www.babelio.com/auteur/Diane-Lavoie/284029": 1,
        "https://www.babelio.com/auteur/Veronique-Cote/226136": 1,
        "https://www.babelio.com/auteur/Charlotte-Gingras/54313": 1,
        "https://www.babelio.com/auteur/Michel-Jean/149380": 1,
        "https://www.babelio.com/auteur/Marilyne-Fortin/333365": 1,
        "https://www.babelio.com/auteur/ric-Plamondon/221283": 1,
        "https://www.babelio.com/auteur/Olivia-Tapiero/129690": 1,
        "https://www.babelio.com/auteur/Gilles-Archambault/2653": 1
    },
    "name": 2768,
    "name_by_value": {
        " Simon Roy": 1,
        " Mil\u00e9na Babin": 1,
        " Diane Lavoie": 1,
        " V\u00e9ronique C\u00f4t\u00e9": 1,
        " Charlotte Gingras": 1,
        " Michel Jean": 1,
        " Marilyne Fortin": 1,
        "

### titre
Pour le titre on normalise simplement

In [29]:
for exemple in Babelio_books:
    if "(" in exemple.title_raw and exemple.title:
        print("exemple simple: titre intial:", exemple.title_raw, " titre traité: ", exemple.title)
        break

exemple simple: titre intial:  Ces étrangers du dedans : Une histoire d'écriture migrante au Québec (1937-1997)   titre traité:  ces etrangers du dedans une histoire d ecriture migrante au quebec


### auteur
Pour les auteurs, on normalise en utilisant la fonction de normalisation. Il n'y a pas besoin de traiter plus loin.
On peut remarque que certains livres comportent une quantité conséquente d'auteur

In [30]:
for exemple in Babelio_books:
    good_exemple = False
    if len(exemple.authors) > 5:
        for author in exemple.authors_raw:
            if '-' in author:
                good_exemple = True
    if good_exemple:
        print("auteur(s) initial(s): ", exemple.authors_raw, "auteur(s) traité: ", exemple.authors)
        break

auteur(s) initial(s):  ['Patrick  Senécal', 'Richard  Goulet', 'Nicole  Roberge', 'Claude  Hébert', 'Réjean  Côte', 'Gino  Saint-Germain', 'Maurice  Landry', 'Nathalie  Saint-Germain', "L'abbé Germain  Langelier", 'Carmelle  Bourgeois-Pelletier', 'Solange  Lebel', 'Valérie  Leclair', 'Francis  Bruneau', 'Mélodie  Roy', 'Luc  LeClerc', 'François  Desfossés', 'Yves  Boisvert', 'Denise  Lizotte', 'Lorraine  Gingras'] auteur(s) traité:  ['patrick senecal', 'richard goulet', 'nicole roberge', 'claude hebert', 'rejean cote', 'gino saint-germain', 'maurice landry', 'nathalie saint-germain', 'abbe germain langelier', 'carmelle bourgeois-pelletier', 'solange lebel', 'valerie leclair', 'francis bruneau', 'melodie roy', 'luc leclerc', 'francois desfosses', 'yves boisvert', 'denise lizotte', 'lorraine gingras']


### isbn
Pour Babelio, on sait que l'isbn est unique et de la forme isbn 13 (car EAN)

In [31]:
for exemple in Babelio_books:
    good_exemple = False
    for isbn in exemple.isbns_raw:
        if len(isbn.replace("\"", "")) == 13 and isbn.replace("\"", "")[-1] != 'X':
            good_exemple = True
    if good_exemple:
        print("exemple d'isbn 13 normalisé: \nisbn initial(s): ", exemple.isbns_raw, "isbn traité: ", exemple.isbns)
        break

exemple d'isbn 13 normalisé: 
isbn initial(s):  ['9782361393090'] isbn traité:  ['9782361393090']


# Comparaison des livres

In [32]:
all_books = generate_all_books(save=False)

def compare_book_on_title(book1, book2):
    if book1.title and book2.title:
        title_eq = Equivalence.PAS_EQUIVALENT
        if Levenshtein.distance(book1.title, book2.title) < max(1, min(len(book1.title), len(book2.title)) / 4):
            title_eq = Equivalence.EQUIVALENT
    else:
        title_eq = Equivalence.PAS_INFO
    return title_eq


def compare_book_on_isbns(book1, book2):
    # Comparaison des isbns par intersection des listes
    if book1.isbns and book2.isbns:
        isbn_eq = Equivalence.PAS_EQUIVALENT
        for isbn1 in book1.isbns:
            for isbn2 in book2.isbns:
                if compare_isbn(isbn1, isbn2):
                    isbn_eq = Equivalence.EQUIVALENT
    else:
        isbn_eq = Equivalence.PAS_INFO
    return isbn_eq


def compare_book_on_authors(book1, book2):
    #Comparaison des auteurs par intersection des listes
    if book1.authors and book2.authors:
        author_eq = Equivalence.PAS_EQUIVALENT
        for author1 in book1.authors:
            for author2 in book2.authors:
                if compare_authors(author1, author2):
                    author_eq = Equivalence.EQUIVALENT
    else:
        author_eq = Equivalence.PAS_INFO
    return author_eq

ADP_loading_time:  11.84359860420227
DL_loading_time:  47.14095640182495
ILE_loading time:  4.9068779945373535
Hurtubise_loading time:  0.06313133239746094
Babelio_loading_time time:  0.5107293128967285


## isbn

In [33]:
found_equal, found_non_equals, found_non_info = False, False, False
for book1, book2 in combinations(all_books, 2):
    isbn_eq = compare_book_on_isbns(book1, book2)
    if not found_equal and isbn_eq == Equivalence.EQUIVALENT:
        found_equal = True
        print("exemple de concordance d'isbn => isbn1: ", book1.isbns_raw, " isbn2: ", book2.isbns_raw)
    if not found_non_equals and isbn_eq == Equivalence.PAS_EQUIVALENT:
        found_non_equals = True
        print("exemple de non-concordance d'isbn => isbn1: ", book1.isbns_raw, " isbn2: ", book2.isbns_raw)
    if not found_non_info and isbn_eq == Equivalence.PAS_INFO:
        found_non_info = True
        print("exemple de manque d'infornation => isbn1: ", book1.isbns_raw, " isbn2: ", book2.isbns_raw)
    if found_equal and found_non_equals and found_non_info:
        break

exemple de non-concordance d'isbn => isbn1:  ['"9782896835027"']  isbn2:  ['"0000000064637"']
exemple de manque d'infornation => isbn1:  ['"9782896835027"']  isbn2:  [' (Br.)\n']
exemple de concordance d'isbn => isbn1:  ['"9782896511006"']  isbn2:  ['"9782896511006"']


## auteur

In [34]:
found_equal, found_non_equals, found_non_info = False, False, False
for book1, book2 in combinations(all_books, 2):
    author_eq = compare_book_on_authors(book1, book2)
    if not found_equal and author_eq == Equivalence.EQUIVALENT:
        found_equal = True
        print("exemple de concordance d'auteurs => auteur1: ", book1.authors_raw, " auteur2: ", book2.authors_raw)
    if not found_non_equals and author_eq == Equivalence.PAS_EQUIVALENT:
        found_non_equals = True
        print("exemple de non-concordance d'auteurs => auteur1: ", book1.authors_raw, " auteur2: ", book2.authors_raw)
    if not found_non_info and author_eq == Equivalence.PAS_INFO:
        found_non_info = True
        print("exemple de manque d'infornation => auteur1: ", book1.authors_raw, " auteur2: ", book2.authors_raw)
    if found_equal and found_non_equals and found_non_info:
        break

exemple de manque d'infornation => auteur1:  ['"" ""']  auteur2:  ['MARCEL-MARIE O.P. DESMARAIS']
exemple de non-concordance d'auteurs => auteur1:  ['MARCEL-MARIE O.P. DESMARAIS']  auteur2:  ['Eric Pier Sperandio']
exemple de concordance d'auteurs => auteur1:  ['MARCEL-MARIE O.P. DESMARAIS']  auteur2:  ['"Louis" "Desmarais"']


## titres

In [35]:
found_equal, found_non_equals, found_non_info = False, False, False
for book1, book2 in combinations(all_books, 2):
    title_eq = compare_book_on_title(book1, book2)
    if not found_equal and title_eq == Equivalence.EQUIVALENT:
        found_equal = True
        print("exemple de concordance de titre => titre1: ", book1.title_raw, " titre2: ", book2.title_raw)
    if not found_non_equals and title_eq == Equivalence.PAS_EQUIVALENT:
        found_non_equals = True
        print("exemple de non-concordance de titre => titre1: ", book1.title_raw, " titre2: ", book2.title_raw)
    if not found_non_info and title_eq == Equivalence.PAS_INFO:
        found_non_info = True
        print("exemple de manque d'infornation => titre1: ", book1.title_raw, " titre2: ", book2.title_raw)
    if found_equal and found_non_equals and found_non_info:
        break

exemple de non-concordance de titre => titre1:  Beyonders, tome 1  titre2:  Capsules d'optimisme
exemple de manque d'infornation => titre1:  Beyonders, tome 1  titre2:  (ParenthÃ¨ses)
exemple de concordance de titre => titre1:  Grimoire des herbes magiques  titre2:  Grimoire des objets magiques


# Analyse de la décision
## Comparaison des livres sur plusieurs critères

In [36]:
found = {}
for case in Case:
    found[case] = False
for book1, book2 in combinations(all_books, 2):
    results = compare_books(book1, book2)
    if results == Case.ISBN_TITRE_EQ and found[results] == False:
        print("\nisbn egaux et titre similaire")
        print(book1, book2)
    if results == Case.ISBN_EQ_TITRE_DIFF and found[results] == False:
        print("\nisbn egaux mais titre legerement differents")
        print(book1, book2)
    if results == Case.ISBN_EQ_TITRE_T_DIFF and found[results] == False:
        print("\nisbn egaux mais titre très differents")
        print(book1, book2)
    if results == Case.ISBN_DIFF_TITRE_EQ and found[results] == False:
        print("\nisbn different mais titres equivalents")
        print(book1, book2)
    if results == Case.ISBN_DIFF_TITRE_AUTEUR_EQ and found[results] == False:
        print("\nisbn different mais titre et auteurs egaux")
        print(book1, book2)
    if results == Case.TITRE_AUTEUR_EQ and found[results] == False:
        print("\ntitre et auteurs egaux")
        print(book1, book2)
    if results == Case.TITRE_EQ_AUTEUR_DIFF and found[results] == False:
        print("\ntitre equivalents mais auteurs differents")
        print(book1, book2)
    if results == Case.TITRE_EQ and found[results] == False:
        print("\ntitre egaux mais pas d'infos en plus")
        print(book1, book2)
    if results == Case.PAS_INFO and found[results] == False:
        print("\nLes informations sont manquantes, pas de prise de décision")
        print(book1, book2)
    found[results] = True
    if all(found.values()):
        break


Les informations sont manquantes, pas de prise de décision
BOOK <http://banq.qc.ca/livre/374249> de Depot_legal Valide:
    titre: Beyonders, tome 1 => beyonders tome 1
    auteur (x0): ['"" ""'] => []
    isbn (x1): ['"9782896835027"'] => ['9782896835027']
 BOOK <http://www.sogides.com/livre/118611> de ADP Valide:
    titre: Capsules d'optimisme => capsules d optimisme
    auteur (x1): ['MARCEL-MARIE O.P. DESMARAIS'] => ['marcel-mariep desmarais']
    isbn (x1): ['"0000000064637"'] => ['0000000064637']


isbn different mais titre et auteurs egaux
BOOK <http://www.sogides.com/livre/395161> de ADP Valide:
    titre: Grimoire des herbes magiques => grimoire des herbes magiques
    auteur (x1): ['Eric Pier Sperandio'] => ['eric pier sperandio']
    isbn (x1): ['"9782764021255"'] => ['9782764021255']
 BOOK <http://www.sogides.com/livre/297468> de ADP Valide:
    titre: Grimoire des objets magiques => grimoire des objets magiques
    auteur (x1): ['Eric Pier Sperandio'] => ['eric pier sper

## Comparaison basée sur le titre

In [37]:
for threshold in range(2, 10):
    for book1, book2 in combinations(all_books, 2):
        if book1.title and book2.title:
            if Levenshtein.distance(book1.title, book2.title) == min(len(book1.title), len(book2.title)) / threshold:
                print("\nmaximum titre equivalent atteint pour 1/", threshold, " de la distance de Levenshtein\n")
                print(book1, book2)
                break


maximum titre equivalent atteint pour 1/ 2  de la distance de Levenshtein

BOOK <http://banq.qc.ca/livre/374249> de Depot_legal Valide:
    titre: Beyonders, tome 1 => beyonders tome 1
    auteur (x0): ['"" ""'] => []
    isbn (x1): ['"9782896835027"'] => ['9782896835027']
 BOOK 9782896476039 de Hurtubise Valide:
    titre: FÃ©licitÃ© - Tome 1 => felicite - tome 1
    auteur (x1): ['Jean-Pierre Charland (De (auteur))'] => ['jean-pierre charland']
    isbn (x3): ['9782896476039', '9782896476220', '9782896478569'] => ['9782896476039', '9782896476220', '9782896478569']


maximum titre equivalent atteint pour 1/ 3  de la distance de Levenshtein

BOOK 28557 de ILE Valide:
    titre: Ã la recherche du soi  => a la recherche du soi
    auteur (x1): [' Ginette Bureau\\'] => ['ginette bureau']
    isbn (x1): ['978-2-923705-25-5\n'] => ['9782923705255']
 BOOK <http://banq.qc.ca/livre/129730> de Depot_legal Valide:
    titre: À la recherche du contraire => a la recherche du contraire
    auteur